Muzhou Liu muzhouliu@wustl.edu 465729

Yidi Zhang yidi.zhang@wustl.edu 465621

In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf 
import sklearn as sk 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from scipy import stats
from scipy.stats import norm, skew 
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import PolynomialFeatures

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
color = sns.color_palette()
sns.set_style('darkgrid')

In [2]:
dataset_train = pd.read_csv('../input/train.csv' )
print(dataset_train.head(3))
X = dataset_train.iloc[:, 1:13]
n = X.shape[0]
# print(X.head(3))
Y = dataset_train.iloc[:, 13]
# test train split


X['sin'] = np.sin( X['Aspect'])
X['cos'] = np.cos( X['Aspect'])
X = X.drop(columns =['Aspect'])


X['climate_zone'] = X['Soil_Type'].astype(str).str[0] 
X['geologic'] = X['Soil_Type'].astype(str).str[1] 
X['soil'] = X['Soil_Type'] % 100
X['soil'] = X['soil'].astype('category')
X['climate_zone'] = X['climate_zone'].astype('category')
X['geologic'] = X['geologic'].astype('category')
X = X.drop(columns =['Soil_Type'])


X['Wilderness_Area1'] = X['Wilderness_Area'].astype('category')
X = X.drop(columns =['Wilderness_Area'])

X.iloc[:,:11] = X.iloc[:,:11].astype('float32')






          ID  Elevation       ...        Soil_Type      Cover_Type
0   40387826       2789       ...             4704  Lodgepole Pine
1  139907508       2075       ...             2702  Ponderosa Pine
2  244759849       2476       ...             2705  Ponderosa Pine

[3 rows x 14 columns]


In [3]:
minmax = StandardScaler().fit(X.iloc[:,:11])
X_mm = minmax.transform(X.iloc[:,:11]) 


X.iloc[:,:11] = X_mm

X_mm_oh = pd.get_dummies(X)



y = pd.get_dummies(Y).values



/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
  


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, axis=1))
            / predictions.shape[0])


In [5]:
learning_rate = 0.0005
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 1024 # 1st layer number of neurons
n_hidden_2 = 2048 # 2nd layer number of neurons
n_hidden_3 = 3036
n_hidden_4 = 4096
n_hidden_5 = 1024
n_hidden_6 = 1024
n_hidden_7 = 512
n_hidden_8 = 512
n_hidden_9 = 384
n_hidden_10 = 384
n_hidden_11 = 256
n_input = 51 # data has 51 imput 
n_output =7


X1 = tf.placeholder("float", [None, n_input])
Y1 = tf.placeholder("float", [None, n_output])



layer_1 = tf.layers.dense(X1,n_hidden_1 , tf.nn.tanh)
layer_2 = tf.layers.dense(layer_1, n_hidden_2, tf.nn.tanh)
layer_3 = tf.layers.dense(layer_2, n_hidden_3, tf.nn.tanh)
layer_4 = tf.layers.dense(layer_3, n_hidden_4, tf.nn.tanh)
layer_5 = tf.layers.dense(layer_4, n_hidden_5, tf.nn.tanh)
layer_6 = tf.layers.dense(layer_5, n_hidden_6, tf.nn.tanh)
layer_7 = tf.layers.dense(layer_6, n_hidden_7, tf.nn.tanh)
layer_8 = tf.layers.dense(layer_7, n_hidden_8, tf.nn.tanh)
layer_9 = tf.layers.dense(layer_8, n_hidden_6, tf.nn.tanh)
layer_10 = tf.layers.dense(layer_9, n_hidden_7, tf.nn.tanh)
layer_11 = tf.layers.dense(layer_10, n_hidden_8, tf.nn.tanh)

logits = tf.layers.dense(layer_11,n_output)



loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels=Y1))  # compute cost

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train_op = optimizer.minimize(loss)

predictions = tf.nn.softmax(logits)





sess = tf.Session()                                 # control training and othersb
sess.run(tf.global_variables_initializer())         # initialize var in graph


for step in range(20001):
        _, l, pred = sess.run([train_op, loss, predictions], {X1: X_mm_oh, Y1: y})
        if accuracy(pred,y) >= 99.93:
            break
        if step % 100 == 0:
                            
            print('step:{} loss:{:.6f} accuracy: {:.2f}'.format(
                    step, l, accuracy(pred, y)))



Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

step:0 loss:1.989471 accuracy: 12.89
step:100 loss:0.632131 accuracy: 72.56
step:200 loss:0.546029 accuracy: 76.09
step:300 loss:0.357997 accuracy: 85.05
step:400 loss:0.356696 accuracy: 85.35
step:500 loss:0.199230 accuracy: 92.24
step:600 loss:0.092440 accuracy: 96.64
step:700 loss:0.090330 accuracy: 96.73
step:800 loss:0.170425 accuracy: 93.52
step:900 loss:0.039745 accuracy: 98.85
step:1000 loss:0.019853 accuracy: 99.57
step:1100 loss:0.027846 accuracy: 99.27


In [6]:
dataset_test = pd.read_csv('../input/test.csv' )
print(dataset_train.head(3))
X_test = dataset_test.iloc[:, 1:13]
n = X_test.shape[0]
# print(X.head(3))
# test train split


X_test['sin'] = np.sin( X_test['Aspect'])
X_test['cos'] = np.cos( X_test['Aspect'])
X_test = X_test.drop(columns =['Aspect'])


X_test['climate_zone'] = X_test['Soil_Type'].astype(str).str[0] 
X_test['geologic'] = X_test['Soil_Type'].astype(str).str[1] 
X_test['soil'] = X_test['Soil_Type'] % 100
X_test['soil'] = X_test['soil'].astype('category')
X_test['climate_zone'] = X_test['climate_zone'].astype('category')
X_test['geologic'] = X_test['geologic'].astype('category')
X_test = X_test.drop(columns =['Soil_Type'])


X_test['Wilderness_Area1'] = X_test['Wilderness_Area'].astype('category')
X_test = X_test.drop(columns =['Wilderness_Area'])

X_test.iloc[:,:11] = X_test.iloc[:,:11].astype('float32')


X_test_mm = minmax.transform(X_test.iloc[:,:11]) 

X_test.iloc[:,:11] = X_test_mm
X_test_mm_oh = pd.get_dummies(X_test)


output = sess.run(predictions,{X1:X_test_mm_oh})
output = np.argmax(output,1)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder().fit(['Aspen', 'Cottonwood/Willow', 'Douglas-fir', 'Krummholz', 'Lodgepole Pine', 'Ponderosa Pine', 'Spruce/Fir'])
output = le.inverse_transform(output)
        
        
ids = dataset_test['ID']

output = pd.DataFrame({ 'ID' : ids, 'Cover_Type': output })
output.to_csv('tanh_submission.csv', index = False)



          ID  Elevation       ...        Soil_Type      Cover_Type
0   40387826       2789       ...             4704  Lodgepole Pine
1  139907508       2075       ...             2702  Ponderosa Pine
2  244759849       2476       ...             2705  Ponderosa Pine

[3 rows x 14 columns]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
